In [1]:
import pandas as pd
import numpy as np
import time
import re
import json
import tqdm
import glob
import requests as rq # http requests
import bs4 as bs4 # beautiful soup for parsing

# 1.0 - Data Collect

In [2]:
df = pd.read_json('./parsed_videos.json', lines=True)

In [55]:
## Get all unique links
unique_links = df.link.unique()
print("Lenght unique links:",len(unique_links))
print("Number of duplicated links", len(df)-len(unique_links))

Lenght unique links: 1780
Number of duplicated links 32


In [56]:
url = 'http://www.youtube.com{link}'

In [ ]:
for link in unique_links:
    urll = url.format(link=link)
    print(urll)
    response = rq.get(urll)
    
    link_name = re.search("v=(.*)", link).group(1)
    with open("./raw_data/video_{}.html".format(link_name),'w+') as output:
        output.write(response.text)
    time.sleep(2)

# 2.0 - Data Processing

In [63]:
with open("parsed_video_info.json","w+") as output:
    for video_file in tqdm.tqdm_notebook(sorted(glob.glob("./raw_data/video*"))):
        with open(video_file,"r+") as inp:
            page_html = inp.read() 
            parsed = bs4.BeautifulSoup(page_html,"html.parser") 
            
            class_watch = parsed.find_all(attrs={"class":re.compile(r"watch")}) 
            id_watch = parsed.find_all(attrs={"id":re.compile(r"watch")}) 
            channel = parsed.find_all("a",attrs={"href":re.compile(r"channel")}) 
            meta = parsed.find_all("meta") 
        
            
            data = dict()
            
            for e in class_watch:
                colname ="_".join(e['class'])
                if 'clearfix' in colname: ## Jump condition to clear fix to not create one column per video
                    continue
                data[colname] = e.text.strip()
                
            for e in id_watch:
                colname = e['id']
                data[colname] = e.text.strip()
                
            for e in meta:
                colname = e.get('property')
                if colname is not None:
                    data[colname] = e['content']
                    
            for link_num, e in enumerate(channel):
                data['channel_link_{}'.format(link_num)] = e['href']
            
            
            output.write("{}\n".format(json.dumps(data)))
                    
                    

# 3.0 - Verification

In [3]:
df = pd.read_json('./parsed_video_info.json',lines=True)
df.shape

(1780, 173)

In [14]:
pd.set_option('display.max_columns',174)

In [29]:
selected_columns = ['watch-title','watch-view-count','watch-time-text','content_watch-info-tag-list',
                   'watch7-headline','watch7-user-header','watch8-sentiment-actions','og:image',
                   'og:image:width','og:image:height','og:description','og:video:width','og:video:height',
                   'og:video:tag','channel_link_0']

In [34]:
df[selected_columns].head(2)

,watch-title,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
0,Data Science & Psychology as an approach in Us...,NaN,"Published on Feb 12, 2020",People & Blogs,Data Science & Psychology as an approach in Us...,Humans of Analytics\n\n\n\n\n\n\n\n\n\n\n\n\n\...,NaN,https://i.ytimg.com/vi/--X0GVq5A3U/hqdefault.jpg,480,360,"With the steaming conversations going around, ...",1280.0,720.0,#AnalyticsCommunity #HumansofAnalytics #DataSc...,/channel/UCgOkTQqzOWNxqFpXymQE7RA
1,Machine Learning Course A To Z || Beginner to ...,"172,078 views","Published on Aug 10, 2018",Education,Machine Learning Course A To Z || Beginner to ...,Geek's Lesson\n\n\n\n\n\n\n\n\n\n\n\n\n\nLoadi...,"172,078 views\n\n\n\n\n\n\n\n5,084\n\nLike thi...",https://i.ytimg.com/vi/-58kO_zYUGE/maxresdefau...,1280,720,Welcome to this free online class on machine l...,640.0,360.0,Ai and machine learning course,/channel/UCKXx22vOENUyHrVAADq7Z_g


In [36]:
df[selected_columns].to_feather('raw_data.feather')

In [38]:
df[selected_columns].to_csv('raw_data_no_labels.csv')

In [ ]:
test = pd.read_csv('./raw_data_no_labels.csv')